In [4]:
from pathlib import Path

import numpy as np
import pandas as pd
import seaborn as sns
from scipy.stats import bootstrap
from tqdm import tqdm
import wandb
import tempfile
import os

tqdm.pandas()

In [2]:
api = wandb.Api()

In [16]:
# all old_bench
run_name_to_id = {
    # "retform_gnn": "",
    # "retmass_fp": "",
    # "retform_fp": "",
    "retform_preconly": "b48mjrk4",
    # "retmass_gnn": "",
    "retmass_preconly": "1k6bzqu8"
}

metric_cols = [
    "cos_sim",
    "cos_sim_sqrt",
    "js_sim",
    "test_hit_rate@1",
    "test_hit_rate@5",
    "test_hit_rate@20"
]

In [17]:
run_name_to_df = {}
with tempfile.TemporaryDirectory() as tmp_dp:
    for run_name, run_id in run_name_to_id.items():
        run = api.run(f"adamoyoung/msg/{run_id}")
        df_file = run.file("df_test.pkl")
        df_fn = df_file.name
        df_fp = os.path.join(tmp_dp,df_fn)
        df_file.download(replace=True, root=tmp_dp)
        run_name_to_df[run_name] = pd.read_pickle(df_fp)

In [18]:
run_name_to_df["retform_preconly"].mean()

losses              0.845981
cos_sim             0.154019
js_sim              0.408519
cos_sim_sqrt        0.165258
cos_sim_obj         0.154019
test_hit_rate@1     0.020896
test_hit_rate@5     0.085192
test_hit_rate@20    0.226542
dtype: float64

In [19]:
# put all dfs into one big df
df = []
for method, method_df in run_name_to_df.items():
    method_df["method"] = method
    df.append(method_df)
df = pd.concat(df)
df

,losses,cos_sim,js_sim,cos_sim_sqrt,cos_sim_obj,test_hit_rate@1,test_hit_rate@5,test_hit_rate@20,method
0,1.000000,0.000000,0.306853,0.000000,0.000000,0.0,0.0,1.0,retform_preconly
1,1.000000,0.000000,0.306853,0.000000,0.000000,0.0,0.0,0.0,retform_preconly
2,1.000000,0.000000,0.306853,0.000000,0.000000,0.0,0.0,0.0,retform_preconly
3,1.000000,0.000000,0.306853,0.000000,0.000000,0.0,0.0,1.0,retform_preconly
4,0.139411,0.860589,0.802057,0.730126,0.860589,0.0,1.0,1.0,retform_preconly
...,...,...,...,...,...,...,...,...,...
9949,1.000000,0.000000,0.306853,0.000000,0.000000,0.0,0.0,0.0,retmass_preconly
9950,0.940658,0.059342,0.386376,0.191875,0.059342,0.0,0.0,0.0,retmass_preconly
9951,1.000000,0.000000,0.306853,0.000000,0.000000,0.0,0.0,0.0,retmass_preconly
9952,1.000000,0.000000,0.306853,0.000000,0.000000,0.0,0.0,0.0,retmass_preconly


In [20]:
def calculate_ci(df, metric_cols):

    # Calculate means for all metrics into a single table
    df_mean = df.groupby('method')[metric_cols].mean().round(3)

    # Calculate confidence intervals for all metrics into a single table
    def get_ci(col_vals, confidence_level=0.999, n_resamples=20_000):
        res = bootstrap((col_vals,), np.mean, confidence_level=confidence_level, n_resamples=n_resamples)
        ci = res.confidence_interval
        return f'{ci.low:.2f}-{ci.high:.2f}'
    def get_ci_for_each_col(df_method):
        return df_method.apply(get_ci, axis=0)
    df_ci = df.groupby('method')[metric_cols].progress_apply(lambda df_method: get_ci_for_each_col(df_method))

    # Merge tables with means and confidence intervals
    for col in metric_cols:
        df_mean[col] = df_mean[col].astype(str) + ' (' + df_ci[col] + ')'
    return df_mean

In [21]:
df_ci = calculate_ci(df, metric_cols)

  0%|          | 0/2 [00:00<?, ?it/s]

100%|██████████| 2/2 [01:08<00:00, 34.11s/it]


In [22]:
df_ci

,cos_sim,cos_sim_sqrt,js_sim,test_hit_rate@1,test_hit_rate@5,test_hit_rate@20
method,,,,,,
retform_preconly,0.154 (0.14-0.16),0.165 (0.16-0.17),0.409 (0.40-0.41),0.021 (0.02-0.03),0.085 (0.08-0.09),0.227 (0.21-0.24)
retmass_preconly,0.154 (0.14-0.16),0.165 (0.16-0.18),0.409 (0.40-0.42),0.004 (0.00-0.01),0.017 (0.01-0.02),0.072 (0.06-0.08)
